<a href="https://colab.research.google.com/github/sanandaraj5597/MLFlow/blob/master/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Assignment 3. Firstly, mlflow needs to be installed

In [0]:
pip install mlflow

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

mkdir: cannot create directory ‘/drive’: File exists


In [0]:
!apt-get install ssh tmux vim

In [0]:
#@title
%%writefile /drive/ngrok-ssh/sshd_config
#	$OpenBSD: sshd_config,v 1.101 2017/03/14 07:19:07 djm Exp $

# This is the sshd server system-wide configuration file.  See
# sshd_config(5) for more information.

# This sshd was compiled with PATH=/usr/bin:/bin:/usr/sbin:/sbin

# The strategy used for options in the default sshd_config shipped with
# OpenSSH is to specify options with their default value where
# possible, but leave them commented.  Uncommented options override the
# default value.

#Port 22
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::

#HostKey /etc/ssh/ssh_host_rsa_key
#HostKey /etc/ssh/ssh_host_ecdsa_key
#HostKey /etc/ssh/ssh_host_ed25519_key

# Ciphers and keying
#RekeyLimit default none

# Logging
#SyslogFacility AUTH
#LogLevel INFO

# Authentication:

#LoginGraceTime 2m
#PermitRootLogin prohibit-password
#StrictModes yes
#MaxAuthTries 6
#MaxSessions 10

#PubkeyAuthentication yes

# Expect .ssh/authorized_keys2 to be disregarded by default in future.
#AuthorizedKeysFile	.ssh/authorized_keys .ssh/authorized_keys2

#AuthorizedPrincipalsFile none

#AuthorizedKeysCommand none
#AuthorizedKeysCommandUser nobody

# For this to work you will also need host keys in /etc/ssh/ssh_known_hosts
#HostbasedAuthentication no
# Change to yes if you don't trust ~/.ssh/known_hosts for
# HostbasedAuthentication
#IgnoreUserKnownHosts no
# Don't read the user's ~/.rhosts and ~/.shosts files
#IgnoreRhosts yes

# To disable tunneled clear text passwords, change to no here!
#PasswordAuthentication yes
#PermitEmptyPasswords no

# Change to yes to enable challenge-response passwords (beware issues with
# some PAM modules and threads)
ChallengeResponseAuthentication no

# Kerberos options
#KerberosAuthentication no
#KerberosOrLocalPasswd yes
#KerberosTicketCleanup yes
#KerberosGetAFSToken no

# GSSAPI options
#GSSAPIAuthentication no
#GSSAPICleanupCredentials yes
#GSSAPIStrictAcceptorCheck yes
#GSSAPIKeyExchange no

# Set this to 'yes' to enable PAM authentication, account processing,
# and session processing. If this is enabled, PAM authentication will
# be allowed through the ChallengeResponseAuthentication and
# PasswordAuthentication.  Depending on your PAM configuration,
# PAM authentication via ChallengeResponseAuthentication may bypass
# the setting of "PermitRootLogin without-password".
# If you just want the PAM account and session checks to run without
# PAM authentication, then enable this but set PasswordAuthentication
# and ChallengeResponseAuthentication to 'no'.
UsePAM yes

#AllowAgentForwarding yes
AllowTcpForwarding yes
#GatewayPorts no
X11Forwarding yes
#X11DisplayOffset 10
#X11UseLocalhost yes
#PermitTTY yes
PrintMotd no
#PrintLastLog yes
#TCPKeepAlive yes
#UseLogin no
#PermitUserEnvironment no
#Compression delayed
#ClientAliveInterval 0
#ClientAliveCountMax 3
#UseDNS no
#PidFile /var/run/sshd.pid
#MaxStartups 10:30:100
#PermitTunnel no
#ChrootDirectory none
#VersionAddendum none

# no default banner path
#Banner none

# Allow client to pass locale environment variables
AcceptEnv LANG LC_*

# override default of no subsystems
Subsystem	sftp	/usr/lib/openssh/sftp-server

# Example of overriding settings on a per-user basis
#Match User anoncvs
#	X11Forwarding no
#	AllowTcpForwarding no
#	PermitTTY no
#	ForceCommand cvs server

Overwriting /drive/ngrok-ssh/sshd_config


In [0]:
%%writefile ~/.ssh/authorized_keys
~/.ssh/id_rsa.pub

Writing /root/.ssh/authorized_keys


In [0]:
!cp /drive/ngrok-ssh/sshd_config /etc/ssh/sshd_config
!service ssh restart
!mkdir -p ~/.ssh

In [0]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

In [0]:
!/ngrok authtoken 1bf3fczopcAZh5c5fu3AY9WB2Nn_3dgegCwKE6vJ4pLgyA2mT

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [0]:
%%writefile /drive/ngrok-ssh/ssh.yml
tunnels:
  ssh:
    proto: tcp
    addr: 22

Overwriting /drive/ngrok-ssh/ssh.yml


In [0]:
%%writefile /drive/ngrok-ssh/http5000.yml
tunnels:
  http5000:
    proto: http
    addr: 5000
    inspect: false
    bind_tls: true

Overwriting /drive/ngrok-ssh/http5000.yml


In [0]:
%%writefile /drive/ngrok-ssh/run_ngrok.sh
#!/bin/sh
set -x
/ngrok start --config ~/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout --config /drive/ngrok-ssh/http5000.yml "$@"

Overwriting /drive/ngrok-ssh/run_ngrok.sh


In [0]:
import os
get_ipython().system_raw('bash /drive/ngrok-ssh/run_ngrok.sh ssh http5000 &')

In [0]:
import requests
import urllib.parse

def get_ngrok_info():
  return requests.get('http://localhost:4040/api/tunnels').json()

def get_ngrok_tunnels():
  for tunnel in get_ngrok_info()['tunnels']:
    name = tunnel['name']
    yield name, tunnel

def get_ngrok_tunnel(name):
  for name1, tunnel in get_ngrok_tunnels():
    if name == name1:
      return tunnel

def get_ngrok_url(name, local=False):
  if local:
    return get_ngrok_tunnel(name)['config']['addr']
  else:
    return get_ngrok_tunnel(name)['public_url']

In [0]:
for name, tunnel in get_ngrok_tunnels():
  local_url = get_ngrok_url(name, local=True)
  public_url = get_ngrok_url(name, local=False)
  print('{:12s} {} <-> {}'.format(name, public_url, local_url))

ssh          tcp://0.tcp.ngrok.io:11205 <-> localhost:22
http5000     https://322d4d47.ngrok.io <-> http://localhost:5000


We are now ready to host our mlflow local host 1t port 5000 gloablly. Use the above mentioned http5000 link to get in.

Now let's get started with our nerual network.

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
import mlflow
import time

In [0]:
batch_size = 4

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = torchvision.datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [0]:
# Download and load the test data
testset = torchvision.datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

In [0]:
class Net(nn.Module):
   def __init__(self):
      super(Net,self).__init__()
      self.conv1 = nn.Conv2d(1,16,3)
      self.conv2 = nn.Conv2d(16,16,3)
      self.fc1   = nn.Linear(24*24*16,100,bias=False)
      self.fc2   = nn.Linear(100,10,bias=False)

   def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.view(-1,24*24*16)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

my_net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(my_net.parameters(), lr=0.001, momentum=0.9)

nn.init.xavier_uniform_(my_net.conv1.weight)
nn.init.xavier_uniform_(my_net.conv2.weight)
nn.init.xavier_uniform_(my_net.fc1.weight)
nn.init.xavier_uniform_(my_net.fc2.weight)

In [0]:
print("Beginning Training...")
exp_id = mlflow.create_experiment('fashion_net_train')

for epoch in range(5):
   
   correct_pred_count = 0
   running_loss = 0.0
   for i, data in enumerate(trainloader,0):
      inputs, labels = data
      
      optimizer.zero_grad()

      outputs = my_net(inputs)
      loss = criterion(outputs,labels)
      loss.backward()
      optimizer.step()

      running_loss += loss

      pred_out, pred_labels = torch.max(outputs,1)

      correct_prediction = accuracy_score(labels,pred_labels,normalize=False) 

      correct_pred_count += correct_prediction

      if i%1000 == 999:
         train_accuracy = (correct_pred_count/((i+1)*batch_size)) * 100
         print('[%d, %5d] loss: %.3f train_accuracy: %.3f' % (epoch + 1, i + 1, running_loss / 1000,train_accuracy))
         run_loss = running_loss/1000

         with mlflow.start_run(experiment_id=exp_id,run_name='fashion_net_train'):
            mlflow.log_metric("loss",float(run_loss))
            mlflow.log_metric("Training_accuracy",float(train_accuracy))

         running_loss = 0.0

print("Finished Training!")

In [0]:
print("Started Testing...")

correct_pred_count = 0

for i,data in enumerate(testloader,0):
   inputs, labels = data

   outputs = my_net(inputs)

   pred_out, pred_labels = torch.max(outputs,1)

   correct_prediction = accuracy_score(labels,pred_labels,normalize=False)

   correct_pred_count += correct_prediction

   max_iter = i

test_accuracy = (correct_pred_count/(batch_size*(max_iter+1))) * 100

print ('Test_accuracy: %.3f' % (test_accuracy))

print("Finished Testing!\n\nStarted loading the model...")

path = './fashion_net.pth'
torch.save(my_net.state_dict(),path)

print("Finished loading the model!")

Started Testing...
Test_accuracy: 90.480
Finished Testing!

Started loading the model...
Finished loading the model!


Now, lets get started with Step 2, varying the model parameters first

The below sub-routine is used for inference given a input testloader

In [0]:
def inference(testloader):
  #print("Started Testing...")
  correct_pred_count = 0

  for i,data in enumerate(testloader,0):
    inputs, labels = data

    outputs = my_net(inputs)

    pred_out, pred_labels = torch.max(outputs,1)

    correct_prediction = accuracy_score(labels,pred_labels,normalize=False)

    correct_pred_count += correct_prediction

    max_iter = i

  test_accuracy = (correct_pred_count/(batch_size*(max_iter+1))) * 100

  #print ('Test_accuracy: %.3f' % (test_accuracy))

  #print("Finished Testing!")

  return test_accuracy

A run-time configurable neural network class.

In [0]:
class Net(nn.Module):
   def __init__(self,number_of_filters1,number_of_filters2,filter1_dim,filter2_dim,num_outputs1):
      super(Net,self).__init__()
      self.fc1_input_dim = number_of_filters2*((28-filter1_dim+1-filter2_dim+1)*(28-filter1_dim+1-filter2_dim+1))
      self.conv1 = nn.Conv2d(1,number_of_filters1,filter1_dim)
      self.conv2 = nn.Conv2d(number_of_filters1,number_of_filters2,filter2_dim)
      self.fc1   = nn.Linear((number_of_filters2*((28-filter1_dim+1-filter2_dim+1)*(28-filter1_dim+1-filter2_dim+1))),num_outputs1,bias=True)
      self.fc2   = nn.Linear(num_outputs1,10,bias=True)

   def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.view(-1,self.fc1_input_dim)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

train_time = 0

for i in range(2):
  filter1_dim = 5 + (i*2)
  for j in range (2):
    filter2_dim = 5 + (j*2)
    for k in range(3):
      number_of_filters1 = 16 + (k*2) + 2
      for l in range(3):
        number_of_filters2 = 16 + (l*2) + 2
        for m in range(2):
          num_outputs1 = 100 + (m+1)*10

          my_net = Net(number_of_filters1,number_of_filters2,filter1_dim,filter2_dim,num_outputs1)
          
          criterion = nn.CrossEntropyLoss()
          optimizer = optim.SGD(my_net.parameters(), lr=0.001, momentum=0.9)

          nn.init.xavier_uniform_(my_net.conv1.weight)
          nn.init.xavier_uniform_(my_net.conv2.weight)
          nn.init.xavier_uniform_(my_net.fc1.weight)
          nn.init.xavier_uniform_(my_net.fc2.weight)

          experiment_name = "fashion_net_" + str(number_of_filters1) + "_" + str(filter1_dim) + "_" + str(number_of_filters2) + "_" + str(filter2_dim) + "_" + str(num_outputs1)

          print("Beginning Training for " + str(experiment_name))
          exp_id = mlflow.create_experiment(experiment_name)
          
          for epoch in range (100):
            correct_pred_count = 0
            running_loss = 0.0
            
            for _, data in enumerate(trainloader,0):
              inputs, labels = data
              
              t = time.time()
              optimizer.zero_grad()

              outputs = my_net(inputs)
              loss = criterion(outputs,labels)
              loss.backward()
              optimizer.step()

              running_loss += loss
   
              train_time += time.time() - t
              pred_out, pred_labels = torch.max(outputs,1)

              correct_prediction = accuracy_score(labels,pred_labels,normalize=False) 

              correct_pred_count += correct_prediction

            train_accuracy = (correct_pred_count/60000) * 100
            run_loss = running_loss/60000
            print('[%d] loss: %.3f train_accuracy: %.3f' % (epoch + 1, run_loss,train_accuracy))

            running_loss = 0.0

            test_accuracy = inference(testloader)

            with mlflow.start_run(experiment_id=exp_id,run_name='fashion_net_train'):
              mlflow.log_metric("loss",float(run_loss))
              mlflow.log_metric("Training_epochs",float(train_time))
              mlflow.log_metric("Training_accuracy",float(train_accuracy))
              mlflow.log_metric("Testing_accuracy",float(test_accuracy))

              mlflow.log_param("Learning_rate",0.001)
              mlflow.log_param("Momentum",0.9)

            if train_accuracy >= 90:
              print('Training time for this configuration is: '+str(train_time))
              print('Test accuracy for this configuration is: '+str(test_accuracy))
              train_time = 0
              break

In [0]:
class Net(nn.Module):
   def __init__(self,number_of_filters1,number_of_filters2,filter1_dim,filter2_dim,num_outputs1):
      super(Net,self).__init__()
      self.fc1_input_dim = number_of_filters2*((28-filter1_dim+1-filter2_dim+1)*(28-filter1_dim+1-filter2_dim+1))
      self.conv1 = nn.Conv2d(1,number_of_filters1,filter1_dim)
      self.conv2 = nn.Conv2d(number_of_filters1,number_of_filters2,filter2_dim)
      self.fc1   = nn.Linear((number_of_filters2*((28-filter1_dim+1-filter2_dim+1)*(28-filter1_dim+1-filter2_dim+1))),num_outputs1,bias=True)
      self.fc2   = nn.Linear(num_outputs1,10,bias=True)

   def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.view(-1,self.fc1_input_dim)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

train_time = 0

for k in range(4):
  if k==0:
    learning_rate = 0.01
    momentum = 0.85
  elif k==1:
    learning_rate = 0.0008
    momentum = 0.85
  elif k==2:
    learning_rate = 0.0008
    momentum = 0.8
  else:
    learning_rate = 0.01
    momentum = 0.8

  for j in range (4):
    if j==0:
      batch_size = 10
    elif j==1:
      batch_size = 25
    elif j==2:
      batch_size = 50
    else:
      batch_size = 500

    for i in range(3):

      my_net = Net(20,18,7,5,110)  #Best config from the previous experiment

      init_id = i
      if i==0:  #Identity matrix initialization
       nn.init.kaiming_uniform_(my_net.conv1.weight)
       nn.init.kaiming_uniform_(my_net.conv2.weight)
       nn.init.kaiming_uniform_(my_net.fc1.weight)
       nn.init.kaiming_uniform_(my_net.fc2.weight)
      elif i==1: #Xavier normal initialization
       nn.init.xavier_normal_(my_net.conv1.weight)
       nn.init.xavier_normal_(my_net.conv2.weight)
       nn.init.xavier_normal_(my_net.fc1.weight)
       nn.init.xavier_normal_(my_net.fc2.weight)
      else: #Default init by Pytorch
       default = 1
          
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.SGD(my_net.parameters(), lr=learning_rate, momentum=momentum)

      trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
      testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

      experiment_name = "fashion_net_" + str(learning_rate) + "_" + str(momentum) + "_" + str(batch_size) + "_" + str(init_id)

      print("Beginning Training for " + str(experiment_name))
      exp_id = mlflow.create_experiment(experiment_name)
          
      for epoch in range (100):
        correct_pred_count = 0
        running_loss = 0.0
            
        for _, data in enumerate(trainloader,0):
          inputs, labels = data
              
          t = time.time()
          optimizer.zero_grad()

          outputs = my_net(inputs)
          loss = criterion(outputs,labels)
          loss.backward()
          optimizer.step()

          running_loss += loss
   
          train_time += time.time() - t
          pred_out, pred_labels = torch.max(outputs,1)

          correct_prediction = accuracy_score(labels,pred_labels,normalize=False) 

          correct_pred_count += correct_prediction

        train_accuracy = (correct_pred_count/60000) * 100
        run_loss = running_loss/60000
        print('[%d] loss: %.3f train_accuracy: %.3f' % (epoch + 1, run_loss,train_accuracy))

        running_loss = 0.0

        test_accuracy = inference(testloader)

        with mlflow.start_run(experiment_id=exp_id,run_name='fashion_net_train'):
          mlflow.log_metric("loss",float(run_loss))
          mlflow.log_metric("Training_epochs",float(train_time))
          mlflow.log_metric("Training_accuracy",float(train_accuracy))
          mlflow.log_metric("Testing_accuracy",float(test_accuracy))

          mlflow.log_param("Learning_rate",0.001)
          mlflow.log_param("Momentum",0.9)
          mlflow.log_param("Batch_size",float(batch_size))
          mlflow.log_param("Initialization_method",float(init_id))

        if train_accuracy >= 90:
          print('Training time for this configuration is: '+str(train_time))
          print('Test accuracy for this configuration is: '+str(test_accuracy))
          train_time = 0
          break

Step 3: Pruning

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score 
import copy
import mlflow

class Net(nn.Module):
   def __init__(self):
      super(Net,self).__init__()
      self.conv1 = nn.Conv2d(1,16,3)
      self.conv2 = nn.Conv2d(16,16,3)
      self.fc1   = nn.Linear(24*24*16,100,bias=True)
      self.fc2   = nn.Linear(100,10,bias=True)

   def forward(self,x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = x.view(-1,24*24*16)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

batch_size = 4

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = torchvision.datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Download and load the test data
testset = torchvision.datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

def inference(testloader):
 print("Started Testing...")

 correct_pred_count = 0

 for i,data in enumerate(testloader,0):
    inputs, labels = data

    outputs = my_net(inputs)

    pred_out, pred_labels = torch.max(outputs,1)

    correct_prediction = accuracy_score(labels,pred_labels,normalize=False)

    correct_pred_count += correct_prediction

    max_iter = i

 test_accuracy = (correct_pred_count/(batch_size*(max_iter+1))) * 100

 print ('Test_accuracy: %.3f' % (test_accuracy))

 print("Finished Testing!")

 return test_accuracy

my_net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(my_net.parameters(), lr=0.001, momentum=0.9)

nn.init.xavier_uniform_(my_net.conv1.weight)
nn.init.xavier_uniform_(my_net.conv2.weight)
nn.init.xavier_uniform_(my_net.fc1.weight)
nn.init.xavier_uniform_(my_net.fc2.weight)

break_flag = 0
for epoch in range(100):
   
   correct_pred_count = 0
   running_loss = 0.0
   for i, data in enumerate(trainloader,0):
      inputs, labels = data
      
      optimizer.zero_grad()

      outputs = my_net(inputs)
      loss = criterion(outputs,labels)
      loss.backward()
      optimizer.step()

      running_loss += loss

      pred_out, pred_labels = torch.max(outputs,1)

      correct_prediction = accuracy_score(labels,pred_labels,normalize=False) 

      correct_pred_count += correct_prediction

      if i%1000 == 999:
         train_accuracy = (correct_pred_count/((i+1)*batch_size)) * 100
         print('[%d, %5d] loss: %.3f train_accuracy: %.3f' % (epoch + 1, i + 1, running_loss / 1000,train_accuracy))
         run_loss = running_loss/1000

         running_loss = 0.0

         if train_accuracy >= 90:
            break_flag = 1
            break

   if break_flag == 1:
      break

test_accuracy_prel = inference(testloader)

print('Test accuracy prel is '+str(test_accuracy_prel))

#torch.save(my_net.state_dict(),'./fashion_net.pth')
my_net_prel = Net()
my_net_prel = copy.deepcopy(my_net)

prune_arr = []

for i in range(16):
  min_val = torch.sum(torch.abs(my_net.conv1.weight[0])).item()
  min_idx = 0

  for j in range (16-1): 
    if(min_val > torch.sum(torch.abs(my_net.conv1.weight[i+1])).item()) and (torch.sum(torch.abs(my_net.conv1.weight[i+1])).item() != 0):
      min_idx = i+1
      min_val = torch.sum(torch.abs(my_net.conv1.weight[i+1])).item()
    if(min_val == 0):
      min_idx = i+1
      min_val = torch.sum(torch.abs(my_net.conv1.weight[i+1])).item()

  print('Gonna prune layer '+str(min_idx))
  my_net.conv1.weight[min_idx][0].detach()
  my_net.conv1.bias[min_idx].detach()
  my_net.conv1.bias[min_idx] = 0
  my_net.conv1.weight[min_idx][0] = 0

  prune_arr.append(min_idx)

  my_net.conv1.weight = torch.nn.parameter.Parameter(my_net.conv1.weight)
  my_net.conv1.bias = torch.nn.parameter.Parameter(my_net.conv1.bias)

  for epoch in range(5):
    print('Beginning training for this epoch')
    for i, data in enumerate(trainloader,0):
     inputs, labels = data
     
     optimizer.zero_grad()

     outputs = my_net(inputs)
     loss = criterion(outputs,labels)
     loss.backward()
     optimizer.step()

     for idx in prune_arr:
       my_net.conv1.weight[idx][0] = 0
       my_net.conv1.bias[idx] = 0
       my_net.conv1.weight = torch.nn.parameter.Parameter(my_net.conv1.weight)
       my_net.conv1.bias = torch.nn.parameter.Parameter(my_net.conv1.bias)

    print('Epoch done')

    correct_pred_count = 0
    for i,data in enumerate(testloader,0):
      inputs, labels = data

      outputs = my_net(inputs)
      
      pred_out, pred_labels = torch.max(outputs,1)
      correct_prediction = accuracy_score(labels,pred_labels,normalize=False)
      correct_pred_count += correct_prediction
      max_iter = i
      
      test_accuracy = (correct_pred_count/(4*(max_iter+1))) * 100

    print ('Test_accuracy: %.3f' % (test_accuracy))
    print("Finished Testing!")
    
    experiment_name = 'conv_layer1_prune' + str(epoch)
    exp_id = mlflow.create_experiment(experiment_name)
    with mlflow.start_run(experiment_id=exp_id,run_name='fashion_net_train'):
      mlflow.log_metric("Epoch",float(epoch+1))
      mlflow.log_metric("Prune_channel",float(min_idx))
      mlflow.log_metric("Test_accuracy",float(test_accuracy))

    if test_accuracy - test_accuracy_prel >= 1:
      break

  #test_accuracy = inference(testloader)

  print('Test accuracy is '+str(test_accuracy))

  if test_accuracy - test_accuracy_prel >= 1:
    break

Going to prune convolution layer 2, FC layer 1

In [0]:
prune_arr_1 = []
max_val = 0

my_net = Net()
my_net = copy.deepcopy(my_net_prel)

optimizer = optim.SGD(my_net.parameters(), lr=0.001, momentum=0.9)

test_accuracy_prel = inference(testloader)
print('Before pruning '+str(test_accuracy_prel))

for j in range(16):
  min_val = torch.sum(torch.abs(my_net.conv2.weight[0])).item()
  min_idx = 0

  for i in range (16-1): 
    if(min_val > torch.sum(torch.abs(my_net.conv2.weight[i+1])).item()) and (torch.sum(torch.abs(my_net.conv2.weight[i+1])).item() != 0):
      min_idx = i+1
      min_val = torch.sum(torch.abs(my_net.conv2.weight[i+1])).item()
    if(min_val == 0):
      min_idx = i+1
      min_val = torch.sum(torch.abs(my_net.conv2.weight[i+1])).item()

  print('Gonna prune layer '+str(min_idx))
  my_net.conv2.weight[min_idx].detach()
  my_net.conv2.bias[min_idx].detach()
  my_net.conv2.bias[min_idx] = 0
  my_net.conv2.weight[min_idx] = 0

  my_net.conv2.weight = torch.nn.parameter.Parameter(my_net.conv2.weight)
  my_net.conv2.bias = torch.nn.parameter.Parameter(my_net.conv2.bias)

  for idx in prune_arr:
    my_net.conv1.weight[idx][0].detach()
    my_net.conv1.bias[idx].detach()
    my_net.conv1.bias[idx] = 0
    my_net.conv1.weight[idx][0] = 0
    my_net.conv1.weight = torch.nn.parameter.Parameter(my_net.conv1.weight)
    my_net.conv1.bias = torch.nn.parameter.Parameter(my_net.conv1.bias)

  prune_arr_1.append(min_idx)

  for epoch in range(5):
    print('Beginning training for this epoch')
    for i, data in enumerate(trainloader,0):
     inputs, labels = data
     
     optimizer.zero_grad()

     outputs = my_net(inputs)
     loss = criterion(outputs,labels)
     loss.backward()
     optimizer.step()

     for idx in prune_arr:
       my_net.conv1.weight[idx][0] = 0
       my_net.conv1.bias[idx] = 0
       my_net.conv1.weight = torch.nn.parameter.Parameter(my_net.conv1.weight)
       my_net.conv1.bias = torch.nn.parameter.Parameter(my_net.conv1.bias)

     for idx in prune_arr_1:
       my_net.conv2.weight[idx] = 0
       my_net.conv2.bias[idx] = 0
       my_net.conv2.weight = torch.nn.parameter.Parameter(my_net.conv2.weight)
       my_net.conv2.bias = torch.nn.parameter.Parameter(my_net.conv2.bias)

    print('Epoch done')

    test_accuracy = inference(testloader)

    experiment_name = 'conv_layer2_prune' + str(epoch)
    exp_id = mlflow.create_experiment(experiment_name)
    with mlflow.start_run(experiment_id=exp_id,run_name='fashion_net_train'):
      mlflow.log_metric("Epoch",float(epoch+1))
      mlflow.log_metric("Prune_channel",float(min_idx))
      mlflow.log_metric("Test_accuracy",float(test_accuracy))

    if test_accuracy - test_accuracy_prel >= 1:
      break

  print('Test accuracy is '+str(test_accuracy))

  if test_accuracy - test_accuracy_prel >= 1:
    break

In [0]:
prune_arr_2 = []
max_val = 0

my_net = Net()
my_net = copy.deepcopy(my_net_prel)

print(my_net.conv1.weight.requires_grad)
print(my_net_prel.conv1.weight.requires_grad)

optimizer = optim.SGD(my_net.parameters(), lr=0.001, momentum=0.9)

test_accuracy_prel = inference(testloader)
print('Before pruning '+str(test_accuracy_prel))

for j in range(100):
  min_val = torch.sum(torch.abs(my_net.fc1.weight[0])).item()
  min_idx = 0

  for i in range (100-1): 
    if(min_val > torch.sum(torch.abs(my_net.fc1.weight[i+1])).item()) and (torch.sum(torch.abs(my_net.fc1.weight[i+1])).item() != 0):
      min_idx = i+1
      min_val = torch.sum(torch.abs(my_net.fc1.weight[i+1])).item()
    if(min_val == 0):
      min_idx = i+1
      min_val = torch.sum(torch.abs(my_net.fc1.weight[i+1])).item()

  print('Gonna prune layer '+str(min_idx))
  my_net.fc1.weight[min_idx].detach()
  my_net.fc1.bias[min_idx].detach()
  my_net.fc1.bias[min_idx] = 0
  my_net.fc1.weight[min_idx] = 0

  my_net.fc1.weight = torch.nn.parameter.Parameter(my_net.fc1.weight)
  my_net.fc1.bias = torch.nn.parameter.Parameter(my_net.fc1.bias)

  for idx in prune_arr:
    my_net.conv1.weight[idx][0].detach()
    my_net.conv1.bias[idx].detach()
    my_net.conv1.bias[idx] = 0
    my_net.conv1.weight[idx][0] = 0
    my_net.conv1.weight = torch.nn.parameter.Parameter(my_net.conv1.weight)
    my_net.conv1.bias = torch.nn.parameter.Parameter(my_net.conv1.bias)
  
  for idx in prune_arr_1:
    my_net.conv2.weight[idx].detach()
    my_net.conv2.bias[idx].detach()
    my_net.conv2.bias[idx] = 0
    my_net.conv2.weight[idx] = 0
    my_net.conv2.weight = torch.nn.parameter.Parameter(my_net.conv2.weight)
    my_net.conv2.bias = torch.nn.parameter.Parameter(my_net.conv2.bias)

  prune_arr_2.append(min_idx)

  for epoch in range(5):
    print('Beginning training for this epoch')
    for i, data in enumerate(trainloader,0):
     inputs, labels = data
     
     optimizer.zero_grad()

     outputs = my_net(inputs)
     loss = criterion(outputs,labels)
     loss.backward()
     optimizer.step()

     for idx in prune_arr_2:
       my_net.fc1.weight[idx]=0
       my_net.fc1.bias[idx]=0
       my_net.fc1.weight = torch.nn.parameter.Parameter(my_net.fc1.weight)
       my_net.fc1.bias = torch.nn.parameter.Parameter(my_net.fc1.bias)

     for idx in prune_arr:
       my_net.conv1.weight[idx][0] = 0
       my_net.conv1.bias[idx] = 0
       my_net.conv1.weight = torch.nn.parameter.Parameter(my_net.conv1.weight)
       my_net.conv1.bias = torch.nn.parameter.Parameter(my_net.conv1.bias)

     for idx in prune_arr_1:
       my_net.conv2.weight[idx] = 0
       my_net.conv2.bias[idx] = 0
       my_net.conv2.weight = torch.nn.parameter.Parameter(my_net.conv2.weight)
       my_net.conv2.bias = torch.nn.parameter.Parameter(my_net.conv2.bias)

    print('Epoch done')

    test_accuracy = inference(testloader)

    experiment_name = 'fc_layer1_prune' + str(epoch)
    exp_id = mlflow.create_experiment(experiment_name)
    with mlflow.start_run(experiment_id=exp_id,run_name='fashion_net_train'):
      mlflow.log_metric("Epoch",float(epoch+1))
      mlflow.log_metric("Prune_channel",float(min_idx))
      mlflow.log_metric("Test_accuracy",float(test_accuracy))

    if test_accuracy - test_accuracy_prel >= 1:
      break

  print('Test accuracy is '+str(test_accuracy))

  if test_accuracy - test_accuracy_prel >= 1:
    break

Pruned Inference vs non-pruned inference

In [0]:
import time

cum_time = 0

inp = torch.rand(1,28,28)
conv1_out = torch.rand(1,16,26,26)
conv2_out = torch.rand(1,16,24,24)
fc1_out = torch.rand(100)
fc2_out = torch.rand(10)

#Freeing the gradient buffers for inference
my_net.conv1.weight.requires_grad = False
my_net.conv2.weight.requires_grad = False
my_net.fc1.weight.requires_grad = False
my_net.fc2.weight.requires_grad = False

my_net.conv1.bias.requires_grad = False
my_net.conv2.bias.requires_grad = False
my_net.fc1.bias.requires_grad = False
my_net.fc2.bias.requires_grad = False

for i in range (16):
    for j in range(26):
      for k in range(26):
        conv1_out[0][i][j][k] = 0
        for l in range(3):
          for m in range(3):
            t = time.time()
            conv1_out[0][i][j][k] += my_net.conv1.weight[i][0][l][m] * inp[0][j+l][k+m]
            conv1_out[0][i][j][k] += my_net.conv1.bias[i]
            cum_time += time.time() - t

for i in range (16):
    for n in range(16):
      for j in range(24):
        for k in range(24):
          conv2_out[0][i][j][k] = 0
          for l in range(3):
            for m in range(3):
              t = time.time()
              conv2_out[0][i][j][k] += my_net.conv2.weight[i][n][l][m] * conv1_out[0][n][j+l][k+m]
              conv2_out[0][i][j][k] += my_net.conv2.bias[i]
              cum_time += time.time() - t

conv2_out_flattened = conv2_out.view(-1)

for i in range(100):
    fc1_out[i] = 0
    for j in range(9216):
      t = time.time()
      fc1_out[i] += my_net.fc1.weight[i][j]*conv2_out_flattened[j]
      fc1_out[i] += my_net.fc1.bias[i]
      cum_time += time.time() - t

for i in range(10):
  fc2_out[i] = 0
  for j in range(100):
    t = time.time()
    fc2_out[i] += my_net.fc2.weight[i][j]*fc1_out[j]
    fc2_out[i] += my_net.fc2.bias[i]
    cum_time += time.time() - t

print('The inference time before pruning is: '+str(cum_time))

cum_time = 0

for i in range (16):
  if i not in prune_arr:
    for j in range(26):
      for k in range(26):
        conv1_out[0][i][j][k] = 0
        for l in range(3):
          for m in range(3):
            t = time.time()
            conv1_out[0][i][j][k] += my_net.conv1.weight[i][0][l][m] * inp[0][j+l][k+m]
            conv1_out[0][i][j][k] += my_net.conv1.bias[i]
            cum_time += time.time() - t

for i in range (16):
  if i not in prune_arr_1:
    for n in range(16):
      for j in range(24):
        for k in range(24):
          conv2_out[0][i][j][k] = 0
          for l in range(3):
            for m in range(3):
              t = time.time()
              conv2_out[0][i][j][k] += my_net.conv2.weight[i][n][l][m] * conv1_out[0][n][j+l][k+m]
              conv2_out[0][i][j][k] += my_net.conv2.bias[i]
              cum_time += time.time() - t

conv2_out_flattened = conv2_out.view(-1)

for i in range(100):
  if i not in prune_arr_2:
    fc1_out[i] = 0
    for j in range(9216):
      t = time.time()
      fc1_out[i] += my_net.fc1.weight[i][j]*conv2_out_flattened[j]
      fc1_out[i] += my_net.fc1.bias[i]
      cum_time += time.time() - t

for i in range(10):
  fc2_out[i] = 0
  for j in range(100):
    t = time.time()
    fc2_out[i] += my_net.fc2.weight[i][j]*fc1_out[j]
    fc2_out[i] += my_net.fc2.bias[i]
    cum_time += time.time() - t

print('The inference time after pruning is: '+str(cum_time))

The inference time before pruning is: 95.12167453765869
The inference time after pruning is: 91.91962003707886
